In [50]:
from pprint import *
from ib_insync import *
util.startLoop()  # uncomment this line when in a notebook

ib = IB() # making a IB class
# ib.connect('127.0.0.1', 7497, clientId=5)
ib.connect('10.0.0.209', 7497, clientId=1)

<IB connected to 10.0.0.209:7497 clientId=1>

In [51]:
f = Future(symbol='MES', lastTradeDateOrContractMonth='202406', exchange='CME', currency='USD') # valid instrument - Future
contract = ib.qualifyContracts(f)[0]
pprint(contract)

Future(conId=620731036, symbol='MES', lastTradeDateOrContractMonth='20240621', multiplier='5', exchange='CME', currency='USD', localSymbol='MESM4', tradingClass='MES')


In [52]:
# first, get a market order

mkt_ord = MarketOrder('BUY', totalQuantity=10)
mkt_trades = ib.placeOrder(contract, mkt_ord)

# print('#' * 100)
ib.sleep(2)
# pprint(mkt_trades) #等2秒过后，这个时候的order已经被fill了，所以打印出来的信息，就已经包括了所有的order的信息了

print('#' * 100)
print('Fill price is: ' + str(mkt_trades.fills[0].execution.price))


# second, get a STOP sell order

stp_ord = StopOrder(action='SELL', totalQuantity=10, stopPrice=mkt_trades.fills[0].execution.price - 10, tif='GTC', outsideRth=True)
stp_trades = ib.placeOrder(contract, stp_ord)
# pprint(stp_trades) #看看这个order的stop lost price对不对


# third get a profit taken limit order for 75-80% of the quantity

lmt_ord = LimitOrder(action='SELL', totalQuantity=8, lmtPrice=mkt_trades.fills[0].execution.price + 10)
lmt_trades = ib.placeOrder(contract, lmt_ord)

# pprint(trades) #看看这个order的profit taken price对不对
ib.sleep(2)


####################################################################################################
Fill price is: 5274.0


True

In [39]:
# 查看一下现在active的order

open_ord = ib.reqOpenOrders()

# pprint(open_ord)

for ord in open_ord:
    # print(ord.order)
    if ord.order.orderType == 'STP':
        existing_stp_ord = ord
        print('Stop loss order -----> ' + str(ord.order.totalQuantity) + ' at the price: ' + str(ord.order.auxPrice))
        # print(ord.order.orderId)
        stp_ord_id = ord.order.orderId
        # print(ord.orderStatus)
        stp_ord_status = ord.orderStatus.status
        # print('#' * 100)
    if ord.order.orderType == 'LMT':
        existing_lmt_ord = ord
        print('Profit taken order -----> ' + str(ord.order.totalQuantity) + 'at the price: ' + str(ord.order.lmtPrice))
        # print(ord.order.orderId)
        lmt_ord_id = ord.order.orderId
        # print(ord.orderStatus)
        lmt_ord_status = ord.orderStatus.status
        # print('#' * 100)

print('&&' * 100)
pprint(existing_stp_ord)
print('#' * 100)
pprint(existing_lmt_ord)
print('#' * 100)
print('STP Order ----->' + ' Order ID: ' + str(stp_ord_id) + '     Order Status: ' + stp_ord_status)
print('LMT Order ----->' + ' Order ID: ' + str(lmt_ord_id) + '     Order Status: ' + lmt_ord_status )

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Trade(contract=Future(conId=620731036, symbol='MES', lastTradeDateOrContractMonth='20240621', multiplier='5', exchange='CME', currency='USD', localSymbol='MESM4', tradingClass='MES'), order=StopOrder(orderId=5, clientId=3, permId=1143270171, action='SELL', totalQuantity=10.0, lmtPrice=0.0, auxPrice=5245.5, tif='GTC', outsideRth=True), orderStatus=OrderStatus(orderId=5, status='Filled', filled=10.0, remaining=0.0, avgFillPrice=5242.0, permId=1143270171, parentId=0, lastFillPrice=5240.25, clientId=3, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Future(conId=620731036, symbol='MES', lastTradeDateOrContractMonth='20240621', multiplier='5', exchange='CME', currency='USD', localSymbol='MESM4', tradingClass='MES'), execution=Execution(execId='000130e8.65fa133a.01.01', time=datetime.datetim

In [93]:
def check_open_order():
    open_ord = ib.reqOpenOrders()

    for ord in open_ord:
        if ord.order.orderType == 'STP':
            existing_stp_ord = ord
            print('STP Order ----->' + '\n'
                    '\t' + 'Order ID: ' + str(ord.order.orderId) + '\n'
                    '\t' + 'Order Status: ' + str(ord.orderStatus.status) +  '\n'
                    '\t' + 'Quantities: ' + str(ord.order.totalQuantity) +  '\n'
                    '\t' + 'At the price: ' + str(ord.order.auxPrice) + '\n')
            print('#' * 100)

        if ord.order.orderType == 'LMT':
            existing_lmt_ord = ord
            print('LMT Order ----->' + '\n'
                    '\t' + 'Order ID: ' + str(ord.order.orderId) + '\n'
                    '\t' + 'Order Status: ' + str(ord.orderStatus.status) +  '\n'
                    '\t' + 'Quantities: ' + str(ord.order.totalQuantity) +  '\n'
                    '\t' + 'At the price: ' + str(ord.order.lmtPrice) + '\n')
            print('#' * 100)

    return(existing_stp_ord, existing_lmt_ord)



if __name__ == '__main__':

    current_open_order = check_open_order()

    pprint(current_open_order)


STP Order ----->
	Order ID: 5
	Order Status: PreSubmitted
	Quantities: 10.0
	At the price: 5264.0

####################################################################################################


UnboundLocalError: local variable 'existing_lmt_ord' referenced before assignment